In [ ]:
import pickle
import os
import us
import numpy as np
import pandas as pd
# pd.set_option('display.max_rows', None)
import matplotlib.pyplot as plt

In [ ]:
weights = pd.read_csv('../data/ref/parsed_usa_intnt_users.csv')

In [ ]:
class Transform:
    # Requirements:
        # import os
        # import pickle
        # import numpy as np
        # import pandas as pd
        # import matplotlib.pyplot as plt
    
    def __init__(self, keyword):
        self.keyword = keyword

    def agg(self, df, kind, **kwargs):
        if (kind=='arithmetic mean'):
            return df.mean(axis=0)
        
        if kind in ['weighted mean', 'weighted']:
            temp = df.merge(kwargs['weights'], how='left', on=kwargs['key'])
            temp['intnt_pop_norm'] = temp['intnt_pop'].apply(lambda x: x / temp['intnt_pop'].sum())
            for col in df.convert_dtypes().select_dtypes(include=['int', 'float']).columns:
                temp[col] = temp[col] * temp['intnt_pop_norm']
            temp = temp.drop(['intnt_pop_norm', 'intnt_pop'], axis=1)
            
            if kind == 'weighted':
                weighted = temp.melt(id_vars=[self.geoName])
                weighted['combine'] = weighted['variable'].str.replace(' ', '_') \
                                                          .str.lower() + '_' + weighted[self.geoName]
                return weighted[['combine', 'value']].round(3).set_index('combine')
            
            if kind == 'weighted mean':
                if self.grouping == 'none':
                    return temp.mean(numeric_only=True)
                return temp.groupby('region').mean()
            
        
    def cleanColumns(self, df, grouping):
        df.columns = df.columns.str.replace('\"', '').str.replace(' ', '_')
        df['date'] = pd.to_datetime(df['date'])
        df = df.sort_values(by='date').reset_index(drop=True)
        
        # Reordering columns to have 'date' as the first column
        col_order = ['date'] + (df.columns.tolist()[:-1])
        df = df[col_order]
        
        if self.geoName == 'state':
            return df
        if self.geoName == 'country_code':
            new_cols = ['date']
            for col in df.columns[1:]:
                if grouping == 'none':
                    brand, denom = col.lower().split('_')
                    new_col = denom + '_' + brand
                else:
                    region, brand, denom = col.lower().split('_')
                    new_col = denom + '_' + brand + '_' + region
                new_cols.append(new_col)                
            new_df = df.rename(columns=dict(zip(df.columns,new_cols)))
            return new_df
    
    def aggFromPickles(self, inputPath, geoName, aggFunc, grouping=pd.DataFrame({'None': [0,0]}), **kwargs):
        # grouping(pd.DataFrame()): dataframe denoting regions to be grouped by, defaults to 'None'
        # aggFunc(str): aggregation method of choice (e.g. 'arithmetic mean')
        df = pd.DataFrame()
        for root, _, files in os.walk(inputPath):
            for file in sorted(files):
                if file.endswith(".pkl"):
                    year = file.split('-')[0]
                    month = file.split('-')[1]
                    with open(os.path.join(root, file), 'rb') as f:
                        df_temp = pickle.load(f)
                
                
                if geoName == 'state':
                    self.geoName = 'state'
                    df_temp = df_temp.rename(columns={'geoName': geoName})
                    conv = us.states.mapping('name', 'abbr')
                    df_temp[geoName] = df_temp[geoName].map(conv)
                    
                else:
                    self.geoName = 'country_code'
                    df_temp = df_temp.rename(columns={'geoName': geoName})
                    df_temp = df_temp.loc[df_temp[geoName]!='U.S. Outlying Islands', :]
                    df_temp[geoName] = cc.convert(names=df_temp[geoName].tolist(),
                                                         to='ISO3',
                                                         not_found='not there')
                    df_temp = df_temp.loc[df_temp[geoName]!='not there', :]
                
                
                if grouping.equals(pd.DataFrame({'None': [0,0]})):
                    self.grouping = 'none'
                    if aggFunc in ['weighted', 'weighted mean']:
                        df_temp = pd.DataFrame(self.agg(df_temp,
                                                        aggFunc,
                                                        weights=kwargs['weights'],
                                                        key=kwargs['key'])).T
                    else:
                        df_temp = pd.DataFrame(self.agg(df_temp.iloc[:, 1:], aggFunc)).T
                    
                else:
                    df_temp_merged = df_temp.merge(grouping, on=geoName)
                    
                    if aggFunc == 'weighted mean':
                        df_temp = self.agg(df_temp_merged,
                                           aggFunc,
                                           weights=kwargs['weights'],
                                           key=kwargs['key'])
                        df_temp = df_temp.reset_index() \
                                         .melt(id_vars='region')
                    else:
                        df_temp = df_temp_merged.groupby(['region']) \
                                                .apply(self.agg, aggFunc) \
                                                .reset_index() \
                                                .melt(id_vars='region')
                    df_temp['Group'] = df_temp['region'] + '_' + df_temp['variable']
                    df_temp = df_temp.rename(columns={'value': aggFunc})
                    df_temp = df_temp[['Group', aggFunc]].set_index('Group').T
                
                df_temp['date'] = year + '-' + month
                df = df.append(df_temp)
        
        df = self.cleanColumns(df, self.grouping)
        self.df = df
        
    def lineplot(self):
        fig, ax = plt.subplots(figsize=(10,7))
        for i in range(1, len(self.df.columns)):
            ax.plot(self.df.iloc[:,0], self.df.iloc[:, i], label=self.df.columns[i])
        # plt.xticks(np.arange(0, len(self.df), 12), 
                   # list(map(str, self.df['date'].dt.year.unique())), 
                   # rotation=60)
        ax.set_ylim(bottom=0)
        ax.set_xlabel('Year')
        ax.set_ylabel('Mean')
        ax.set_title('Keyword: \'{}\''.format(self.keyword.capitalize()))
        ax.legend()
    
    def toPickle(self, outputPath):
        fileName = 'global-arimean-{}'.format(self.keyword)
        pathName = '{}/{}.pkl'.format(outputPath, fileName)
        with open(pathName, 'wb') as f:
            pickle.dump(self.df, f)

    def toCSV(self, outputPath, aggFunc, grouping):
        fileName = '{}-{}-{}'.format(self.keyword, aggFunc, grouping)
        pathName = '{}/{}.csv'.format(outputPath, fileName)
        self.df.to_csv(pathName, index=False)

class Transform_keyword(Transform):
# Subclass of class Transform
    def __init__(self, keyword):
        super().__init__(keyword)

    def path(self):
        return '../data/raw/{}'.format(self.keyword)

    def aggFromPickles(self):
        super().aggFromPickles(self.path())

In [ ]:
t = Transform('cuisine')

In [ ]:
t.aggFromPickles(inputPath='../data/raw',
                 geoName='state',
                 aggFunc='weighted',
                 weights= weights,
                 key='state')

In [ ]:
t.df

In [ ]:
t.toCSV('../data/agg', 'wtd', 'usa')